# Imports


In [ ]:
import warnings
import h3
import ee
import google
import os
import folium

import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Polygon
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tobler.util import h3fy

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
try:
  import ee_jupyter
except ModuleNotFoundError:
  print('ee_jupyter was not found. Installing now...')
  result = os.system('pip -q install earthengine-jupyter')
  import ee_jupyter
print(f'ee_jupyter (version {ee_jupyter.__version__}) '
        f'is installed.')

In [ ]:
from ee_jupyter.ipyleaflet import Map

In [ ]:
warnings.filterwarnings('ignore')

# Import Temperature Data and Set Up Map

In [ ]:
# kelvin air temperature for Feburary 1, 7 AM
kelvin_air_temp = ee.Image('ECMWF/ERA5_LAND/HOURLY/20230201T07').select('temperature_2m')
farenheit_air_temp = kelvin_air_temp.multiply(1.8).subtract(459.67)

In [ ]:
location_lonlat= [-134.424637, 58.301041]
map_init_params = {
    'center': list(reversed(location_lonlat)), # <lat,lon> ordering
    'zoom': 3
}
map1 = Map(**map_init_params)

In [ ]:
map1.addLayer(
  kelvin_air_temp,
  {'min': 233, 'max': 270, 'palette': ['grey', 'blue']}, # remember temperature here is in Kelvin
  'air temperature (Kelvin)'
)

In [ ]:
map1.addLayer(
  farenheit_air_temp,
  {'min': -40, 'max': 26, 'palette': ['white', 'green']}, # remember temperature here is in Farenheit
  'air temperature (Farenheit)'
)

# Census Tract (2020)

In [ ]:
tracts_feature = ee.FeatureCollection("TIGER/2020/TRACT")
ak_tracts = tracts_feature.filter(ee.Filter.equals('STATEFP', '02'))

In [ ]:
ak_tracts.size().getInfo()

In [ ]:
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': '00000080'}
map1.addLayer(ak_tracts.style(**style), {}, "Census Tracts")

In [ ]:
map1

In [ ]:
reduced_temp_tract = kelvin_air_temp.reduceRegions(ak_tracts, ee.Reducer.mean())

In [ ]:
print(type(reduced_temp_tract))
print(type(reduced_temp_tract.first()))
# find the smallest and biggest man temperature across alaska for this time period
print(reduced_temp_tract.aggregate_max('mean').getInfo())
print(reduced_temp_tract.aggregate_min('mean').getInfo())

In [ ]:
tract_map = Map(**map_init_params)

empty = ee.Image().byte()
fills_tract = empty.paint(
  featureCollection = reduced_temp_tract,
  color = 'mean',
)
tract_map.addLayer(fills_tract, {'palette': ['white', 'red'], 'min': 247, 'max': 277.5}, 'mean temp by census tract')

In [ ]:
tract_map

In [ ]:
task = ee.batch.Export.table.toDrive(**{
  'collection': reduced_temp_tract,
  'description': 'Tract Average Temp',
  'folder': 'EarthEngine',
  'fileFormat': 'CSV'
})
task.start()

# Census Boroughs (2018)

In [ ]:
census_boroughs = ee.FeatureCollection("TIGER/2018/Counties")
ak_boroughs = census_boroughs.filter(ee.Filter.equals('STATEFP', '02'))

In [ ]:
ak_boroughs.size().getInfo()

In [ ]:
reduced_temp_borough = kelvin_air_temp.reduceRegions(ak_tracts, ee.Reducer.mean())

In [ ]:
borough_map = Map(**map_init_params)

empty = ee.Image().byte()
fills_borough = empty.paint(
  featureCollection = reduced_temp_borough,
  color = 'mean',
)
borough_map.addLayer(fills_borough, {'palette': ['white', 'red'], 'min': 247, 'max': 277.5}, 'mean temp by borough')

In [ ]:
borough_map

In [ ]:
task = ee.batch.Export.table.toDrive(**{
  'collection': reduced_temp_borough,
  'description': 'Borough Average Temp',
  'folder': 'EarthEngine',
  'fileFormat': 'CSV'
})
task.start()

# Census Block Groups (2020)

This is a little slower than the above calculations, but not too bad.

In [ ]:
census_bg = ee.FeatureCollection("TIGER/2020/BG")
ak_bg = census_bg.filter(ee.Filter.equals('STATEFP', '02'))
ak_bg.size().getInfo()

In [ ]:
reduced_temp_bg = kelvin_air_temp.reduceRegions(ak_bg, ee.Reducer.mean())

In [ ]:
bg_map = Map(**map_init_params)

empty = ee.Image().byte()
fills_bg = empty.paint(
  featureCollection = reduced_temp_bg,
  color = 'mean',
)
bg_map.addLayer(fills_bg, {'palette': ['white', 'red'], 'min': 247, 'max': 277.5}, 'mean temp by census block group')

In [ ]:
bg_map

In [ ]:
task = ee.batch.Export.table.toDrive(**{
  'collection': reduced_temp_bg,
  'description': 'Block Group Average Temp',
  'folder': 'EarthEngine',
  'fileFormat': 'CSV'
})
task.start()

# Hexagons!!

In [ ]:
latitude_fairbanks = 64.837776
longitude_fairbanks = -147.716385
hex_id = h3.geo_to_h3(latitude_fairbanks, longitude_fairbanks, resolution=4)
geometry = Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True))

In [ ]:
# get the following shape file from
# https://live.laborstats.alaska.gov/cen/maps/gis/Boroughs2020.zip
# hosted at
# https://live.laborstats.alaska.gov/cen/maps-gis

geodf = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Boroughs2020/Boroughs2020.shp')
hexgrid_res5 = h3fy(geodf.buffer(0.01), resolution=5)
hexgrid_res5.plot(figsize=(15,15))

In [ ]:
hexgrid_res4 = h3fy(geodf.buffer(0.01), resolution=4)
hexgrid_res4.plot(figsize=(15,15))

In [ ]:
hexgrid_res4.size

In [ ]:
hexgrid_res5.size

In [ ]:
geodf = geodf.to_crs(4326)
hexgrid_res5_4326 = h3fy(geodf.buffer(0.01), resolution=5)
hexgrid_res4_4326 = h3fy(geodf.buffer(0.01), resolution=4)

In [ ]:
def get_ee_feature(geom):
    x,y = geom.exterior.coords.xy
    coords = np.dstack((x,y)).tolist()
    g = ee.Geometry.Polygon(coords)
    return ee.Feature(g)
features5 = ee.FeatureCollection([get_ee_feature(x[1].geometry) for x in hexgrid_res5_4326.iterrows()])
features4 = ee.FeatureCollection([get_ee_feature(x[1].geometry) for x in hexgrid_res4_4326.iterrows()])

In [ ]:
image_collection = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY")
image = image_collection.filterDate('2012-01-01', '2022-12-31').select('temperature_2m').mean().multiply(1.8).subtract(459.67)


In [ ]:
image.reduceRegions(collection=features4, reducer=ee.Reducer.mean(), scale=11132, tileScale=1).size().getInfo()

In [ ]:
zone_stats4 = image.reduceRegions(collection=features4, reducer=ee.Reducer.mean(), scale=11132, tileScale=1).getInfo()
zone_stats4 = gpd.GeoDataFrame.from_features(zone_stats4, crs='epsg:4326')
zone_stats4_3338 = zone_stats4.to_crs(3338)


In [ ]:
# the following plot is wacky due to CRS; Alaska crosses the international date line
fig, ax = plt.subplots(figsize=(15,15))

# Define the legend axis
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%",  pad=0.1)

# Plot the U.S. map with hexagons, where the colour represents the mean air temperature
zone_stats4.plot(column='mean', ax=ax, cmap='OrRd', legend=True, cax=cax)

# Customise the legend
cax.minorticks_on()
cax.tick_params(which='major', direction='in', length=18, width=2, labelsize=24)
cax.tick_params(which='minor', direction='in', length=12, width=2, labelsize=24)
cax.set_title('Temperature (K)', fontsize=24)

# Turn the x-axis and y-axis off
ax.set_axis_off()

In [ ]:
zone_stats4_3338 = zone_stats4.to_crs(3338)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

# Define the legend axis
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%",  pad=0.1)

# Plot the U.S. map with hexagons, where the colour represents the mean air temperature
zone_stats4_3338.plot(column='mean', ax=ax, cmap='OrRd', legend=True, cax=cax)

# Customise the legend
cax.minorticks_on()
cax.tick_params(which='major', direction='in', length=18, width=2, labelsize=24)
cax.tick_params(which='minor', direction='in', length=12, width=2, labelsize=24)
cax.set_title('Temperature (K)', fontsize=24)

# Turn the x-axis and y-axis off
ax.set_axis_off()

Visualizing Resolution 5 is not possible due to the number of hexagons.

In [ ]:
temp_map = zone_stats4_3338.explore(column='mean', name='Mean Temperature', cmap='OrRd', 
                              style_kwds={'opacity':0.4, 'color':'black', 'opacity':0.2}, 
                              highlight_kwds={'fillOpacity':1}, 
                              tooltip_kwds={'aliases':['Temperature']}, 
                              legend_kwds={'caption':'Mean Temperature (F)'})

# Add layer control to the map
folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(temp_map)
folium.TileLayer('cartodbpositron',overlay=True,name="View in Light Mode").add_to(temp_map)
folium.LayerControl(collapsed=False).add_to(temp_map)

temp_map